In [ ]:
from google.colab import files
uploaded = files.upload()

Saving moby_dick_four_chapters.txt to moby_dick_four_chapters.txt


In [ ]:
import spacy
from string import punctuation
import numpy as np


In [ ]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text 

print(read_file('moby_dick_four_chapters.txt')[:1000])

Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in this.  If they but
knew it,

In [ ]:
nlp = spacy.load('en', disable = ['parser', 'tagger', 'ner'])
nlp.max_length = 1198626
punctuation = punctuation + '\n' + ' ' + '\n\n' + ' ' + '\n\n\n' + ' -- '

In [ ]:
def seperate_punctuation(document):

  return [token.text.lower() for token in nlp(document) if token.text not in punctuation]


In [ ]:
text = read_file('moby_dick_four_chapters.txt')
tokens = seperate_punctuation(text)
len(tokens)

11326

In [ ]:
def create_sequences(sequence_length, tokens):

  train_length = sequence_length + 1
  text_sequences = []

  for token in range(train_length,len(tokens)):

    sequence = tokens[token-train_length:token]
    text_sequences.append(sequence)

  return text_sequences  

text_sequences = create_sequences(25, tokens) # !!!

In [ ]:
print(text_sequences[0])
print('\n')
print(text_sequences[1])
print('\n')
print(text_sequences[2])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']


['me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore']


['ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i']


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

Using TensorFlow backend.


In [ ]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [ ]:
sequences = np.array(sequences)
vocab_size = len(tokenizer.word_counts)

In [ ]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size + 1)
sequence_length = X.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

In [ ]:
def create_model(vocab_size,sequence_length):

  model = Sequential()
  model.add(Embedding(vocab_size, sequence_length ))
  model.add(LSTM(150, return_sequences=True))
  model.add(LSTM(150, return_sequences=True))
  model.add(LSTM(150))
  model.add(Dense(50, activation='relu'))

  model.add(Dense(vocab_size, activation='softmax'))

  model.compile(loss ='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  model.summary()

  return model

In [ ]:
model = create_model(vocab_size + 1, sequence_length)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 25)          67925     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 150)         105600    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 150)         180600    
_________________________________________________________________
lstm_3 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                7550      
_________________________________________________________________
dense_2 (Dense)              (None, 2717)              138567    
Total params: 680,842
Trainable params: 680,842
Non-trainable params: 0
________________________________________________

In [ ]:
from pickle import dump, load
model.fit(X,y, batch_size = 64, epochs = 200, verbose = 1)

In [ ]:
model.save('mobydick_model.h5')
dump(tokenizer, open('mobydick_tokenizer', 'wb'))

In [ ]:
from keras.preprocessing.sequence import pad_sequences

def generate_text(model, tokenizer, sequence_length, seed_text, number_of_words):

  output_text = [] 
  input_text = seed_text
  
  for i in range(number_of_words):

    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = pad_sequences([encoded_text], maxlen = sequence_length,truncating = 'pre') 
    pred_wordIndex = model.predict_classes(pad_encoded, verbose=0)[0]
    predicted_word = tokenizer.index_word[pred_wordIndex]
    input_text += ' ' + predicted_word
    output_text.append(predicted_word)

  return ' '.join(output_text)



In [ ]:
import random
random.seed(101)
random_selection = random.randint(0, len(text_sequences))
random_seed = text_sequences[random_selection]
seed_text = ' '.join(random_seed)

In [ ]:
print(seed_text)

i am he has just as much reason to fear me as i have to be afraid of him better sleep with a sober cannibal than


In [ ]:
#50 Layers Model
generate_text(model, tokenizer, sequence_length, seed_text=seed_text, number_of_words=25)

'a drunken harpooneer with this glass underneath--"the soul nay you a green box coat addressed \'s the embalmed fetching the landlord of a course of'

In [ ]:
sentence = "It is a nice feeling on the beach when a little wind goes between your head and the sun above you"

In [ ]:
#50 Layers Model
generate_text(model, tokenizer, sequence_length, seed_text=sentence, number_of_words=25)

'week week week week market market market everything everything family e e e e e e e e e e e e e e e'

In [ ]:
#150 Layers Model
generate_text(model, tokenizer, sequence_length, seed_text=seed_text, number_of_words=25)

'fowls;--though fowls;--though fowls;--though bedfellow bedfellow bedfellow bedfellow tapping tapping tapping tapping tapping put put round round round round round round heath.--it heath.--it heath.--it heath.--it heath.--it'

In [ ]:
#150 Layers Model
generate_text(model, tokenizer, sequence_length, seed_text=sentence, number_of_words=25)